# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 10 2022 12:36PM,250504,10,0086157512,ISDIN Corporation,Released
1,Nov 10 2022 12:36PM,250504,10,0086157513,ISDIN Corporation,Released
2,Nov 10 2022 12:36PM,250504,10,0086157516,ISDIN Corporation,Released
3,Nov 10 2022 12:36PM,250504,10,0086157517,ISDIN Corporation,Released
4,Nov 10 2022 12:36PM,250504,10,0086157519,ISDIN Corporation,Released
5,Nov 10 2022 12:36PM,250504,10,0086157518,ISDIN Corporation,Released
6,Nov 10 2022 12:36PM,250504,10,0086157520,ISDIN Corporation,Released
7,Nov 10 2022 12:36PM,250504,10,0086157521,ISDIN Corporation,Released
8,Nov 10 2022 12:36PM,250504,10,0086157523,ISDIN Corporation,Released
9,Nov 10 2022 12:36PM,250504,10,0086157522,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,250492,Released,62
11,250499,Released,7
12,250500,Executing,1
13,250502,Released,12
14,250504,Released,60


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250492,NaN,NaN,62.0
250499,NaN,NaN,7.0
250500,NaN,1.0,NaN
250502,NaN,NaN,12.0
250504,NaN,NaN,60.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250363,0.0,1.0,28.0
250416,54.0,24.0,5.0
250436,0.0,0.0,1.0
250437,0.0,0.0,6.0
250442,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250363,0,1,28
250416,54,24,5
250436,0,0,1
250437,0,0,6
250442,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250363,0,1,28
1,250416,54,24,5
2,250436,0,0,1
3,250437,0,0,6
4,250442,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250363,,1,28
1,250416,54,24,5
2,250436,,,1
3,250437,,,6
4,250442,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation
60,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC"
72,Nov 10 2022 12:01PM,250500,12,Hush Hush
73,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC"
80,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation
142,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc."
143,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc."
172,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC
173,Nov 9 2022 4:13PM,250442,10,"Senseonics, Incorporated"
174,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,,,60
1,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",,,12
2,Nov 10 2022 12:01PM,250500,12,Hush Hush,,1,
3,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",,,7
4,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,,,62
5,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc.",,1,
6,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc.",,1,28
7,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
8,Nov 9 2022 4:13PM,250442,10,"Senseonics, Incorporated",,,1
9,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60,,
1,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",12,,
2,Nov 10 2022 12:01PM,250500,12,Hush Hush,,1,
3,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",7,,
4,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,62,,
5,Nov 10 2022 11:04AM,250490,20,"ACI Healthcare USA, Inc.",,1,
6,Nov 10 2022 10:57AM,250363,15,"Alliance Pharma, Inc.",28,1,
7,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
8,Nov 9 2022 4:13PM,250442,10,"Senseonics, Incorporated",1,,
9,Nov 9 2022 3:04PM,250437,10,"Methapharm, Inc.",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60,,
1,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",12,,
2,Nov 10 2022 12:01PM,250500,12,Hush Hush,,1,
3,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",7,,
4,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,62,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60.0,NaN,NaN
1,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",12.0,NaN,NaN
2,Nov 10 2022 12:01PM,250500,12,Hush Hush,NaN,1.0,NaN
3,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",7.0,NaN,NaN
4,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,62.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 10 2022 12:36PM,250504,10,ISDIN Corporation,60.0,0.0,0.0
1,Nov 10 2022 12:04PM,250502,15,"Virtus Pharmaceuticals, LLC",12.0,0.0,0.0
2,Nov 10 2022 12:01PM,250500,12,Hush Hush,0.0,1.0,0.0
3,Nov 10 2022 11:57AM,250499,15,"Mizner Bioscience, LLC",7.0,0.0,0.0
4,Nov 10 2022 11:18AM,250492,10,ISDIN Corporation,62.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1252311,130.0,0.0,0.0
12,250500,0.0,1.0,0.0
15,1252255,53.0,25.0,54.0
20,250490,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1252311,130.0,0.0,0.0
1,12,250500,0.0,1.0,0.0
2,15,1252255,53.0,25.0,54.0
3,20,250490,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,130.0,0.0,0.0
1,12,0.0,1.0,0.0
2,15,53.0,25.0,54.0
3,20,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,130.0
1,12,Released,0.0
2,15,Released,53.0
3,20,Released,0.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,20
Status,,,,
Completed,0.0,0.0,54.0,0.0
Executing,0.0,1.0,25.0,1.0
Released,130.0,0.0,53.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,20
0,Completed,0.0,0.0,54.0,0.0
1,Executing,0.0,1.0,25.0,1.0
2,Released,130.0,0.0,53.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,20
0,Completed,0.0,0.0,54.0,0.0
1,Executing,0.0,1.0,25.0,1.0
2,Released,130.0,0.0,53.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()